In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
import pandas as pd
from scipy.special import softmax
import csv
import urllib.request

# Load CSV file and create DataFrame

In [2]:
data = pd.read_csv("DiscordData.csv")
data.head()

,AuthorID,Author,Date,Content,Attachments,Reactions
0,370963041141129226,Daiigr#7964,02-Jul-20 08:53 PM,who wants to play something?,NaN,NaN
1,607796009707372554,Emo Knightmare#5231,02-Jul-20 10:17 PM,Sorry fam no gaming after 8 PM courtesy of my ...,NaN,NaN
2,370963041141129226,Daiigr#7964,03-Jul-20 12:26 AM,Oof,NaN,NaN
3,537937247891357697,The Dark Soul#7970,03-Jul-20 12:31 AM,big sad,NaN,NaN
4,503221991717863424,LET ME IN!!!#0749,03-Jul-20 01:14 AM,If anyone here plays DBD and wants to have som...,NaN,NaN


In [3]:
print(data['Author'].unique())

['Daiigr#7964' 'Emo Knightmare#5231' 'The Dark Soul#7970'
 'LET ME IN!!!#0749' 'PETTAPLAY#7395' 'kima#1596' 'haani_05#5594'
 'Deleted User#0000' 'existential.duck#5117' 'Eeugh#0033'
 'Master chief#7460' 'KyroGaming#1987' 'soumar#3751' 'Iann#4893'
 'Haruka Sakura#2591' 'schrimpf#2940' 'Dwonks#8334' 'Rythm#3722'
 'hagha#2602' 'carlitos#1736' 'rY4n#8975' 'seafox2205#6878' 'MEE6#4876'
 'Daiigr AI#4911' 'Father#2152' 'XxEXO_TROOPERxX#8603'
 "I'll do my best#9945" 'Simple Poll#9879' 'Peachymango11#3226'
 'Minecaft Bot#1886' 'lana#8237' 'sofia719462#2756' 'UnoBot#6134'
 '(G)raveyard #dear-god#0000' 'Son#9990' 'Loli Police#5803'
 'Dank Memer#5192' 'Lord Lazinga#7942' 'neurotell#9825' 'i have name#0161'
 'Cool Frog Facts #frogs#0000' 'Caitlin#6324']


In [4]:
cleaned_data = data

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'Rythm#3722'].index)

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'Loli Police#5803'].index)

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'Cool Frog Facts #frogs#0000'].index)

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'Minecaft Bot#1886'].index)

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'Dank Memer#5192' ].index)

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'Simple Poll#9879' ].index)

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'UnoBot#6134' ].index)

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'MEE6#4876' ].index)

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'Daiigr AI#4911' ].index)



print(cleaned_data['Author'].unique())

print(len(data))
print(len(cleaned_data))

['Daiigr#7964' 'Emo Knightmare#5231' 'The Dark Soul#7970'
 'LET ME IN!!!#0749' 'PETTAPLAY#7395' 'kima#1596' 'haani_05#5594'
 'Deleted User#0000' 'existential.duck#5117' 'Eeugh#0033'
 'Master chief#7460' 'KyroGaming#1987' 'soumar#3751' 'Iann#4893'
 'Haruka Sakura#2591' 'schrimpf#2940' 'Dwonks#8334' 'hagha#2602'
 'carlitos#1736' 'rY4n#8975' 'seafox2205#6878' 'Father#2152'
 'XxEXO_TROOPERxX#8603' "I'll do my best#9945" 'Peachymango11#3226'
 'lana#8237' 'sofia719462#2756' '(G)raveyard #dear-god#0000' 'Son#9990'
 'Lord Lazinga#7942' 'neurotell#9825' 'i have name#0161' 'Caitlin#6324']
187600
184938


# Preprocess text (username and link placeholders)


In [5]:
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
emoji, emotion, hate, irony, offensive, sentiment
stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary


In [6]:
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [7]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

# download label mapping

In [8]:
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

In [9]:
def listToString(instr):
        emptystr=""
        for ele in instr: 
            emptystr += ele + " "
        return emptystr

In [16]:
Content = cleaned_data['Content'].copy().sample(n=500)
Content = Content.values

In [18]:
StrContent = []
for line in Content:
    StrContent.append(str(line))

Scores_arr = []
for i in StrContent:
    text = i
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    try:
        output = model(**encoded_input)
    except:
        print('Error')

    scores = output[0][0].detach().numpy()
    Scores_arr.append(softmax(scores))

    print('i=' +str(len(Scores_arr)))
  

i=1
i=2
i=3
i=4
i=5
i=6
i=7
i=8
i=9
i=10
i=11
i=12
i=13
i=14
i=15
i=16
i=17
i=18
i=19
i=20
i=21
i=22
i=23
i=24
i=25
i=26
i=27
i=28
i=29
i=30
i=31
i=32
i=33
i=34
i=35
i=36
i=37
i=38
i=39
i=40
i=41
i=42
i=43
i=44
i=45
i=46
i=47
i=48
i=49
i=50
i=51
i=52
i=53
i=54
i=55
i=56
i=57
i=58
i=59
i=60
i=61
i=62
i=63
i=64
i=65
i=66
i=67
i=68
i=69
i=70
i=71
i=72
i=73
i=74
i=75
i=76
i=77
i=78
i=79
i=80
i=81
i=82
i=83
i=84
i=85
i=86
i=87
i=88
i=89
i=90
i=91
i=92
i=93
i=94
i=95
i=96
i=97
i=98
i=99
i=100
i=101
i=102
i=103
i=104
i=105
i=106
i=107
i=108
i=109
i=110
i=111
i=112
i=113
i=114
i=115
i=116
i=117
i=118
i=119
i=120
i=121
i=122
i=123
i=124
i=125
i=126
i=127
i=128
i=129
i=130
i=131
i=132
i=133
i=134
i=135
i=136
i=137
i=138
i=139
i=140
i=141
i=142
i=143
i=144
i=145
i=146
i=147
i=148
i=149
i=150
i=151
i=152
i=153
i=154
i=155
i=156
i=157
i=158
i=159
i=160
i=161
i=162
i=163
i=164
i=165
i=166
i=167
i=168
i=169
i=170
i=171
i=172
i=173
i=174
i=175
i=176
i=177
i=178
i=179
i=180
i=181
i=182
i=183
i=184
i=18

In [19]:
for i in Scores_arr:
    i[0] *= -1 

In [20]:

df = pd.DataFrame(Scores_arr, columns = ['Negative', 'Neutral', 'Positive'])
df.head()

,Negative,Neutral,Positive
0,-0.010101,0.257149,0.732750
1,-0.886514,0.103248,0.010239
2,-0.014549,0.220191,0.765260
3,-0.001541,0.020780,0.977679
4,-0.778359,0.191689,0.029952


In [21]:
import plotly.express as px


fig = px.scatter(df, y=['Negative','Neutral','Positive'],template = "plotly_dark",trendline="lowess")
fig.show() 

In [22]:
fig = px.scatter(df, trendline="lowess",template = "plotly_dark",
                title="sentiment analysis of discord server data")
fig.data = [t for t in fig.data if t.mode == "lines"]
fig.update_traces(showlegend=True) #trendlines have showlegend=False by default
fig.show()